## Librairies utiles

# Breast cancer Wisconsin

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Le dataset Breast Cancer Wisconsin

Le dataset est accessible sur :  
https://www.kaggle.com/uciml/breast-cancer-wisconsin-data  
http://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29  
(on peut utiliser pd.read_table pour lire un fichier .dat)

In [ ]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

On peut afficher les 10 premières lignes du dataset :

In [ ]:
df.head(10)

On a les informations suivantes :
1) ID number 2) Diagnosis (M = malignant, B = benign) 3-32)

Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter)  
b) texture (standard deviation of gray-scale values)  
c) perimeter  
d) area  
e) smoothness (local variation in radius lengths)  
f) compactness (perimeter^2 / area - 1.0)  
g) concavity (severity of concave portions of the contour)  
h) concave points (number of concave portions of the contour)  
i) symmetry  
j) fractal dimension ("coastline approximation" - 1)  
  
The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.  
  
All feature values are recoded with four significant digits.

In [ ]:
df.columns

Pour avoir l'ensemble du tableau, on peut utiliser un affichage au format HTML :

In [ ]:
from IPython.core.display import HTML # permet d'afficher du code html dans jupyter
display(HTML(df.head(10).to_html()))

## Données

df.shape donne la taille du tableau, ici : 569 lignes et 33 colonnes

In [ ]:
df.shape

df.describe donne des éléments statistiques sur chacune des colonnes : moyenne, std, quartiles, etc.

In [ ]:
df.describe()

In [ ]:
df.columns

La colonne **'Unnamed: 32'** est vide : on va la supprimer. Axis = 0 pour supprimer des lignes, axis = 1 pour supprimer des colonnes

In [ ]:
df = df.drop(['Unnamed: 32'], axis=1)

In [ ]:
df.diagnosis.value_counts()

Ici on voit qu'il y a 357 bénins et 212 malins. Diagnosis est la colonne dans laquelle ont été comptées les apparitions pour chaque valeur possible (value_counts)

## Visualisations

malin et benin ci-dessous sont deux booléens : malin correspond à M dans diagnosis et benin à B

In [ ]:
malin = df.diagnosis=='M'
benin = df.diagnosis=='B'

*jointplot* permet de visualiser dans un plan les distributions d'un couple de paramètres :

*kde = kernel density estimation*

In [ ]:
sns.jointplot("perimeter_worst", "area_worst", df, kind='kde');

On voit bien ici la répartition des tumeurs selon le périmètre et la surface. On n'a pas encore distingué les bénins et les malins dans cette visualisation.

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(df.perimeter_worst, df.area_worst,  shade=True)

On peut tracer ce type de graphique avec des couleurs, ce qui nous permet d'intégrer dans le prochain graphique la distinction entre malin et bénin :

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(df[malin].perimeter_worst, df[malin].area_worst, cmap="Reds",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(df[benin].perimeter_worst, df[benin].area_worst, cmap="Greens", shade=True, alpha=0.3, shade_lowest=False)

Ici on déduit donc qu'en moyenne les tumeurs malignes ont un périmètre et une surface plus importantes que les tumeurs bénignes. Ce n'est pas non plus une distinction claire puisqu'il y a une zone où les deux se recouvrent, mais il y a une nette tendance.

Les **diagrammes en boîte** (ou **boîtes à moustaches** ou **box plot**) résument quelques caractéristiques de position du caractère étudié (médiane, quartiles, minimum, maximum ou déciles). Ce diagramme est utilisé principalement pour comparer un même caractère dans deux populations de tailles différentes. Il s'agit de tracer un rectangle allant du premier quartile au troisième quartile et coupé par la médiane. On ajoute alors des segments aux extrémités menant jusqu'aux valeurs extrêmes.  
Par exemple pour la répartion des espèces selon la longueur du sépale :

In [ ]:
sns.boxplot(x="diagnosis", y="perimeter_worst", data=df)

Ici aussi on voit que le périmètre est plus important sur les malins que sur les bénins.

Les **violins plots** sont similaires aux box plots, excepté qu’ils permettent de montrer la courbe de densité de probabilité des différentes valeurs. Typiquement, les violins plots présentent un marqueur pour la médiane des données et l’écart interquartile, comme dans un box plot standard.

In [ ]:
sns.violinplot(x="diagnosis", y="perimeter_worst", data=df)

Cette représentation permet de mettre en évidene que les périmètres des bénins sont globalement plus resserrés autour de la moyenne.

*FacetGrid* permet de superposer des graphiques selon une ou plusieurs caractéristiques. On crée une structure avec *FacetGrid*, et on trace ensuite les graphiques avec *map*

In [ ]:
fig = sns.FacetGrid(df, hue="diagnosis", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "perimeter_worst", shade=True)
fig.add_legend()

On veut tracer un nuage de points selon le rayon et la texture de la tumeur, en différenciant la couleur des points selon le diagnostic :

Ici on change de paramètres et on choisit de travailler sur le rayon et la texture. On reste sur le dataframe df. On indique qu'on veut une teinte différente selon le critère (= la colonne) diagnosis, donc selon M et B. On peut ajouter une droite de régression en mettant "fit_reg=True".

In [ ]:
sns.lmplot(x="radius_mean", y="texture_mean", data=df, fit_reg=False, hue='diagnosis')

*pairplot* affiche les nuages de points associés à tous les couples de paramètres :

In [ ]:
#sns.pairplot(df, hue="diagnosis")

## Machine learning

Maintenant on va essayer de prédire si la tumeur est bénine ou maligne (on passe donc à des algorithmes de machine learning)

On sépare le dataset en deux parties :
- un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
- un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
data_train = df.sample(frac=0.8, random_state=1)          # 80% des données avec frac=0.8
data_test = df.drop(data_train.index)     # le reste des données pour le test

On a donc récupéré 80% des données dans data_train.

Il reste à séparer ce que l'on veut prédire (la cible, ici diagnosis) du reste. Pour cela on distingue X_train et Y_train : dans X_train on prend data_train duquel on drop diagnosis, et dans Y_train on met le reste. On fait la même chose sur data_test.

On sépare les données d'apprentissage (*X_train*) et la cible (*y_train*, la colonnes des données *classe*)

In [ ]:
X_train = data_train.drop(['diagnosis'], axis=1)
y_train = data_train['diagnosis']
X_test = data_test.drop(['diagnosis'], axis=1)
y_test = data_test['diagnosis']

## Régression logistique

On veut prédire une variable aléatoire $Y$ à partir d'un vecteur de variables explicatives $X=(X_1,...,X_n)$
On 


Les variables explicatives sont par exemple la surface, le périmètre, etc. L'idée est d'utiliser la fonction logistique, que l'on trace tout simplement.

La fonction logistique $\frac{e^{x}}{1+e^{x}}$ varie entre $-\infty$ et $+\infty$ pour $x$ variant entre $0$ et $1$.  
Elle est souvent utilisée pour "mapper" une probabilité et un espace réel

L'intérêt de mapper une probabilité ici est que selon la répartition que nous avons vous plus haut, on a vu qu'il y avait une zone de recouvrement entre M et B selon le périmètre. Ainsi, on aura pas la même probabilité que la tumeur soit M si le périmètre est dans la zone où elle peut aussi être B ou si elle est bien au dessus. Plus on est proches de la zone de séparation moins on est sûrs du diagnostique.

La fonction logistique correspond un peu à cette répartition.

In [ ]:
plt.figure(figsize=(9,9))

logistique = lambda x: np.exp(x)/(1+np.exp(x))   

x_range = np.linspace(-10,10,50)       
y_values = logistique(x_range)

plt.plot(x_range, y_values, color="red")

La régression logistique consiste à trouver une fonction linéaire C(X) qui permette d'estimer la probabilité de $Y=1$ connaissant $X$ :
$$p(Y=1|X) = \frac{e^{C(X)}}{1+e^{C(X)}}$$

Autrement dit, cela revient à trouver une séparation linéaire des caractéristiques qui minimise un critère d'erreur.

Pour plus de détails, cf par exemple :  
http://eric.univ-lyon2.fr/~ricco/cours/cours/pratique_regression_logistique.pdf

On peut tracer la courbe de régression logistique pour prédire l'espèce Virginica à partir de la longueur du sépale avec la fonction *lmplot* :

On veut maintenant prédire l'espèce à partir de toutes les caractéristiques, et évaluer la qualité de cette prédiction en utilisant la régression logistique définie dans la librairie *sklearn* :

In [ ]:
from sklearn.linear_model import LogisticRegression

On entraîne le modèle de régression logistique avec *fit* :

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

Ici on a donné un nom (lr) à la régression, et appliqué la méthode fit qui permet de faire l'apprentissage du l'ensemble de train, X et Y.

On peut prédire les valeurs sur l'ensemble de test avec le modèle entraîné :

In [ ]:
y_lr = lr.predict(X_test)

Une fois l'apprentissage fait sur TRAIN, on prédit donc sur TEST. L'idée est de comparer les vrais résultats Y_test (on les a récupérées plus tôt) avec les prédictions faites maintenant sur X_test et stockées dans y_lr. En comptant le nombre d'erreurs, on obtiendra un score d'accuracy.

## Score et matrice de confusion

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

La mesure de pertinence compte le nombre de fois où l'algorithme a fait une bonne prédiction (en pourcentage) :

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

On voit ici qu'on a raison dans 60% des cas, ce qui n'est pas très bon. Contrairement à ce que l'on pourrait penser, le pire score n'est pas d'avoir 0% (= avoir faux tout le temps) car il suffit dans certains cas d'inverser l'hypothèse. Le pire score possible est donc plutôt 50%.

Ce n'est pas la même chose de prédire qu'un patient est malade du coronavirus alors qu'il ne l'est pas et de prédire qu'il n'est pas malade alors qu'il l'est. Ainsi, une mesure plus fine consiste à compter le nombre de **faux positif** (valeur prédite 1 et réelle 0) et de **vrai négatif** (valeur prédite 0 et réelle 1). On utilise une **matrice de confusion** :

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_lr)
print(cm)

<img src="https://i.stack.imgur.com/gKyb9.png">

Dans notre cas on a 45 faux positifs. Normalement il ne devrait y avoir des choses que sur la diagonale, sinon 0.

On peut aussi utiliser la méthode **crosstab** de **Pandas** (plutôt que la méthode confusion_matrix de sklearn) pour afficher la matrice de confusion :

In [ ]:
pd.crosstab(y_test, y_lr, rownames=['Reel'], colnames=['Prediction'], margins=True)

### Exercice  
Le résultat est-il satisfaisant ?  
Quel pourrait être le problème ?

Le résultat n'est pas satisfaisant. Il n'y a aucun vrai négatif, et il y a 45 faux positifs. Un des pb peut être la prise en compte de l'id dans l'apprentissage : il n'influe absolument pas le résultat, et pourtant il est pris en compte alors qu'il est différent pour chaque ligne de données. Une suggestion est donc de drop la colonne id des données d'entrainement.

In [ ]:
df.columns

In [ ]:
df = df.drop(['id'], axis=1)

In [ ]:
df.head()

Une fois la colonne id supprimée, on recommence le dataset.

In [ ]:
data_train = df.sample(frac=0.8, random_state=1)
data_test = df.drop(data_train.index) 
X_train = data_train.drop(['diagnosis'], axis=1)
y_train = data_train['diagnosis']
X_test = data_test.drop(['diagnosis'], axis=1)
y_test = data_test['diagnosis']
lr.fit(X_train,y_train)
y_lr = lr.predict(X_test)
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)
cm = confusion_matrix(y_test, y_lr)
print(cm)

Le score d'accuracy s'est nettement amélioré ! On est maintenant à 96%. 
Concernant la matrice de confusion la situation est également meilleure : on a une large majorité de vrais positifs et vrais négatifs. Il reste 3 faux positifs et 1 faux négatif.

Attention on peut avoir un résultat légèrement différent si on met autre chose que 1 en valeur pour random state dans le data_train. En s'entrainant sur des données différentes, les applications sur le test peut varier.

## Arbres de décision

Au début de l'exercice on a remarqué une potentielle distinction à faire à partir de 110 de périmètre, valeur au delà de laquelle la courbe des tumeurs M passe au dessus de la courbe des B.

In [ ]:
fig = sns.FacetGrid(df, hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "perimeter_worst", shade=True)
fig.add_legend()

Ainsi, l'idée ici est d'éliminer les données ayant un périmètre inférieur à 110, puis de les rediviser en deux groupes (au delà d'une certaine valeur) à partir d'un autre critère. Ici on prendra la texture.

In [ ]:
fig = sns.FacetGrid(df[df.perimeter_worst>110], hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "texture_mean", shade=True)
fig.add_legend()

On remarque que la courbe B passe en dessous de la courbe M lorsque x = 17. On procède donc à la même cission en se concentrant sur les texture_mean au delà de 17.
On va maintenant étudier la concavité pour voir si l'on peut également faire une coupure.

In [ ]:
fig = sns.FacetGrid(df[(df.perimeter_worst>110) & (df.texture_mean>17)], hue="diagnosis", aspect=3) # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "concave points_mean", shade=True)
fig.add_legend()

On constate là aussi une nette chute de la courbe B, qui passe en dessous de la courbe M vers x = 0.06. Si on suit ce type de raisonnement, on aboutit à un arbre de décision.

Un arbre de décision permet de faire à chaque étape un choix entre deux possibilités, pour arriver à une réponse sur les feuilles (cf. Akinator)

Pour construire un arbre de décision à partir d'un ensemble d'apprentissage, on va choisir une variable qui sépare l'ensemble en deux parties les plus distinctes en fonction d'un critère. Sur les iris par exemple, on peut utiliser la largeur du pétale pour séparer l'espèce Setosa des autres.

L'indice *GINI* mesure avec quelle fréquence un élément aléatoire de l'ensemble serait mal classé si son étiquette était sélectionnée aléatoirement depuis la distribution des étiquettes dans le sous-ensemble.

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_dtc = dtc.predict(X_test)
print(accuracy_score(y_test, y_dtc))

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc, feature_names=X_train.columns, class_names=['benin','malin'], fontsize=14, filled=True)  

On peut modifier certains paramètres :  Le paramètre *max_depth* est un seuil sur la profondeur maximale de l’arbre. Le paramètre *min_samples_leaf* donne le nombre minimal d’échantillons dans un noeud feuille.

In [ ]:
dtc1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 20)
dtc1.fit(X_train,y_train)

On obtient un arbre un peu différent :

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc1, feature_names=X_train.columns, class_names=['benin','malin'], fontsize=14, filled=True)  

In [ ]:
y_dtc1 = dtc1.predict(X_test)
print(accuracy_score(y_test, y_dtc1))

Le problème ici est que le choix des critères étudiés (et leur ordre) est un petit peu arbitraire. Donc il existe en fait plusieurs arbres ; on parle alors de forêts aléatoires.

Pour plus de détails sur les arbres de décision :  
https://zestedesavoir.com/tutoriels/962/les-arbres-de-decisions/comprendre-le-concept/#1-les-origines  
http://cedric.cnam.fr/vertigo/Cours/ml2/tpArbresDecision.html  
http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/Slides/coursFM_AD-RF.pdf  

## Random forests

<img src="https://infinitescript.com/wordpress/wp-content/uploads/2016/08/Random-Forest-Example.jpg">

cf par exemple :  
https://fr.wikipedia.org/wiki/For%C3%AAt_d%27arbres_d%C3%A9cisionnels  
https://www.biostars.org/p/86981/  
https://infinitescript.com/2016/08/random-forest/

Il s'agit ici de "comparer" plusieurs arbres et de ne garder que les meilleurs. Pour cela on utilise la méthode RandomForestClassifier

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

Et la matrice de confusion :

In [ ]:
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

Le score est encore un peu meilleur avec une accuracy de 97,3%. On a trois *faux positif*. 
C'est une bonne méthode, assez rapide, pour résoudre des pb.

### Importance des caractéristiques

L'attribut *feature_importances_* renvoie un tableau du poids de chaque caractéristique dans la décision :

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)

On peut visualiser ces degrés d'importance avec un graphique à barres par exemple :

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), df.columns[indices])
plt.title('Importance des caracteristiques')

Il apparait ici que la texture, le périmètre et la dimension fractale sont très importantes dans le diagnostique.

### Exercice : les iris

Le dataset des iris est prédéfini dans seaborn :

In [ ]:
df = sns.load_dataset("iris")

In [ ]:
df.head()

On a les informations suivantes :
- longueur du sépale (en cm)
- largeur du sépale
- longueur du pétale
- largeur du pétale
- espèce : Virginica, Setosa ou Versicolor

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQM3aH4Q3AplfE1MR3ROAp9Ok35fafmNT59ddXkdEvNdMkT8X6E">

In [ ]:
sns.pairplot(df, hue="species")

Appliquer les méthodes de visualisation et de machine learning vues précédemment sur ce dataset

## Données

Le but ici est de déterminer l'espèce en fonction des caractéristiques du dataset. Nous avons 5 colonnes : sepal_length, sepal_width, petal_length, petal_width et species.
Le dataset est composé de 150 lignes de données.
Chaque colonne semble pour le moment pertinente pour la détermination de l'espèce : nous n'en retirons pas du dataset.

In [ ]:
df.columns

In [ ]:
df.shape

Dans le dataset il y a 50 exemplaires de chaque espèce. L'échantillon va donc nous permettre de comparer correctement les données.

In [ ]:
df.species.value_counts()

## Visualisation

Etant donné le faible nombre de critères, il est pertinent de les étudier par paire pour mieux visualiser quels critères nous permettront de différencier les espèces.

In [ ]:
sns.pairplot(df, hue="species")

On remarque ainsi que la setosa se distingue nettement des deux autres sur presque tous les critères. Les versicolor et virginica se recouvrent cependant : il sera moins aisé de les distinguer. On remarque tout de même que la séparation est plus claire sur les critères liés au pétales que sur ceux liés aux sépales.

In [ ]:
se = df.species=='setosa'
ve = df.species=='versicolor'
vi = df.species=='virginica'

plt.figure(figsize=(12,12))
sns.kdeplot(df[se].petal_length, df[se].petal_width, cmap="Greens",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(df[ve].petal_length, df[ve].petal_width, cmap="Reds",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(df[vi].petal_length, df[vi].petal_width, cmap="Blues",  shade=True, alpha=0.3, shade_lowest=False)

On déduit ici qu'en moyenne les virginica ont une longueur et une largeur de pétale plus importantes que les versicolor. Ce n'est pas non plus une distinction claire puisqu'il y a une zone où les deux se recouvrent, mais une nette tendance se dégage : à partir d'une petal_length de 5 il y a plus de chance d'avoir une virginica. Cela est confirmé dans le graphique ci-dessous puisque la courbe des versicolor passe en dessous de la courbe des virginica au delà d'une longueur de pétale de 5.

Nous choisissons ici de comparer le critère petal_length ainsi que sa distribution pour les différentes espèces : les paramètres de la versicolor sont globalement plus resserés autour de la moyenne que la virginica, bien que ce critère ne suffise pas à les distinguer. (Encore une fois, on constate que les petal_length de la setosa sont nettement plus bas - la valeur max est inférieure à la valeur min des deux autres espèces - et que ses valeurs sont très resserrées.)

In [ ]:
sns.violinplot(x="species", y="petal_length", data=df)

In [ ]:
fig = sns.FacetGrid(df, hue="species", aspect=1, palette="Set2")
fig.map(sns.kdeplot, "petal_length", shade=True)
fig.add_legend()

Nous changeons de paramètre et étudions ici les sépales.

In [ ]:
sns.lmplot(x="sepal_length", y="sepal_width", data=df, hue='species', fit_reg=True)

Les droites de régression sont très rapprochées pour les versicolor et pour les virginica : ce critère permettra difficilement de les distinguer, sauf au delà d'une certaine taille. En effet, dans notre échantillon de données, il n'existe pas de versicolor avec des sépales de plus de 7.0 de long et de plus de 3.4 de large.

## Machine learning

Nous allons désormais essayer de prédire l'espèce de l'iris grâce à des algorithmes de machine learning. Pour cela on sépare le dataset en 2 parties : une d'entrainement (80% des données), et une pour tester l'efficacité de notre prédiction (20%).

In [ ]:
data_train = df.sample(frac=0.8, random_state=1)
data_test = df.drop(data_train.index) 

Il reste à séparer ce que l'on veut prédire (la cible, ici l'espèce : species) du reste. Pour cela on distingue X_train et Y_train : dans X_train on prend data_train duquel on drop species, et dans Y_train on met le reste. On fait la même chose sur data_test.

In [ ]:
X_train = data_train.drop(['species'], axis=1)
y_train = data_train['species']
X_test = data_test.drop(['species'], axis=1)
y_test = data_test['species']

L'idée ici est de prédire les valeurs sur X_test (predict) à partir du modèle entraîné sur X_train (fit).

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_lr = lr.predict(X_test)

Ainsi nous pouvons comparer nos résultats de prédiction (y_lr) avec les résultats présents dans notre dataset (Y_test).

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)
cm = confusion_matrix(y_test, y_lr)
print(cm)

Nous obtenons un score de 1, qui est donc un score parfait : l'algorithme a donc fait 100% de bonnes prédictions.
Seule la diagonale de la matrice de confusion ne contient pas des nombres égaux à zéro, ce qui signifie que sur l'échantillon testé il n'y a eu aucun faux positif ou faux négatif.

## Arbre de décision

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(max_depth = 3)
dtc.fit(X_train,y_train)
y_dtc = dtc.predict(X_test)
print(accuracy_score(y_test, y_dtc))

In [ ]:
plt.figure(figsize=(30,30))
tree.plot_tree(dtc, feature_names=X_train.columns, class_names=['setosa', 'versicolor', 'virginica'], fontsize=14, filled=True)

## Random forests

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)